In [1]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
sys.path.append(PROJECT_ROOT)

In [2]:
import json
import yaml

# load config
with open('../configs.yaml', 'r', encoding='utf-8') as f:
    cf = yaml.safe_load(f)

In [3]:
# full train COCO2017 anns
with open(cf['data']['raw_annotations']['train'], 'r', encoding='utf-8') as f:
    train2017 = json.load(f)

# full val COCO2017 anns
with open(cf['data']['raw_annotations']['val'], 'r', encoding='utf-8') as f:
    val2017 = json.load(f)

In [4]:
selected_ids = [5, 6, 7, 17, 18, 22, 24, 25, 63, 65]    # Danh sách các ids categories muốn split

selected_categories  = []
for cat in train2017['categories']:
    if cat['id'] in selected_ids:
        selected_categories.append(cat)

selected_categories

[{'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},
 {'supercategory': 'animal', 'id': 22, 'name': 'elephant'},
 {'supercategory': 'animal', 'id': 24, 'name': 'zebra'},
 {'supercategory': 'animal', 'id': 25, 'name': 'giraffe'},
 {'supercategory': 'furniture', 'id': 63, 'name': 'couch'},
 {'supercategory': 'furniture', 'id': 65, 'name': 'bed'}]

In [5]:
selected_categories_ids = {c["id"] for c in selected_categories}
selected_categories_ids

{5, 6, 7, 17, 18, 22, 24, 25, 63, 65}

In [6]:
def filter_coco(coco, selected_category_ids):
    anns = [
        ann for ann in coco["annotations"]
        if ann["category_id"] in selected_category_ids
    ]

    img_ids = {ann["image_id"] for ann in anns}

    imgs = [
        img for img in coco["images"]
        if img["id"] in img_ids
    ]

    cats = [
        c for c in coco["categories"]
        if c["id"] in selected_category_ids
    ]

    return {
        "images": imgs,
        "annotations": anns,
        "categories": cats
    }


In [7]:
train2017_10cls = filter_coco(train2017, selected_categories_ids)
val2017_10cls = filter_coco(val2017, selected_categories_ids)

In [12]:
print(f"Len train dataset: {len(train2017_10cls['images'])}")
print(f"Len val dataset: {len(val2017_10cls['images'])}")

Len train dataset: 30919
Len val dataset: 1303


In [9]:
# with open('code/object_detection_coco/coco/annotations/instances_train2017_10cls(1).json', 'w', encoding='utf-8') as f:
#     json.dump(train2017_10cls, f, ensure_ascii=False, indent=4)

# with open('code/object_detection_coco/coco/annotations/instances_val2017_10cls(1).json', 'w', encoding='utf-8') as f:
#     json.dump(val2017_10cls, f, ensure_ascii=False, indent=4)

In [10]:
# import json
# import os
# import shutil
# from tqdm import tqdm

# # paths
# json_path = "code/object_detection_coco/coco/annotations/instances_train2017_10cls(1).json"
# src_img_dir = "code/object_detection_coco/coco/train2017"
# dst_img_dir = "code/object_detection_coco/coco/train2017_10cls(1)"

# os.makedirs(dst_img_dir, exist_ok=True)

# # load json
# with open(json_path, "r") as f:
#     coco = json.load(f)

# images = coco["images"]

# # copy images
# for img in tqdm(images):
#     file_name = img["file_name"]
#     src_path = os.path.join(src_img_dir, file_name)
#     dst_path = os.path.join(dst_img_dir, file_name)

#     if not os.path.exists(src_path):
#         print(f"[WARN] Missing image: {file_name}")
#         continue

#     shutil.copy2(src_path, dst_path)
